## Загрузка и предобработка файла

In [1]:
import pandas as pd

https://drive.google.com/file/d/1hAgbwKURl8yLgMDiTrB-T_V5tg3SqQTn/view?usp=sharing

In [2]:
! gdown --id 1hAgbwKURl8yLgMDiTrB-T_V5tg3SqQTn


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1hAgbwKURl8yLgMDiTrB-T_V5tg3SqQTn
To: /content/per_lob.csv
100% 38.8k/38.8k [00:00<00:00, 44.0MB/s]


In [4]:
df = pd.read_csv('/content/per_lob.csv', sep=';')

In [5]:
df.head()

,number,type,tree,bush,diameter,height,description,action
0,1,клен ясенелистный (поросль),NaN,2.0,NaN,"1,5-1,9",ослаблено,Вырубить
1,2,клен ясенелистный (поросль),NaN,1.0,NaN,"1,8",растет из под ограждения,Вырубить
2,3,ива (поросль),NaN,3.0,NaN,"0,5-0,7",угнетено,Вырубить
3,4,ива (поросль),NaN,1.0,NaN,"0,6",угнетено,Вырубить
4,5,клен ясенелистный (поросль),NaN,1.0,NaN,"2,5",ослаблено,Вырубить


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   number       299 non-null    object 
 1   type         299 non-null    object 
 2   tree         272 non-null    float64
 3   bush         27 non-null     float64
 4   diameter     272 non-null    object 
 5   height       299 non-null    object 
 6   description  299 non-null    object 
 7   action       299 non-null    object 
dtypes: float64(2), object(6)
memory usage: 18.8+ KB


В столбцах tree и bush есть пропуски - это специфика данных, в tree отмечено количество только деревьев, bush соответственно кустов

In [7]:
df.tree.value_counts()

1.0     251
2.0      12
3.0       4
4.0       1
5.0       1
19.0      1
6.0       1
20.0      1
Name: tree, dtype: int64

In [8]:
df['tree'] = pd.to_numeric(df['tree'], errors='coerce')
df['bush'] = pd.to_numeric(df['bush'], errors='coerce')


In [9]:
display('Общее количество деревьев и кустарников')
df.groupby('action', as_index=False)[['tree', 'bush']].sum()

'Общее количество деревьев и кустарников'

,action,tree,bush
0,Вырубить,341.0,888.0


##Получение данных

Тут только вырубка (еще бывает пересадка и сохранение). Значит для расчетов нам нужны следующие данные: 
1. количество поросли и кустарников, 
2. количество деревьев твердолиственных и мягколиственных, 
3. количество деревьев по группам (диаметр от 1 до 10, от 11 до 20 и тд), 
4. является ли дерево многоствольным, число стволов, 
5. диаметр пня. Если указно несколько диаметров, берем бОльший. Диаметр пня у многоствольных больше. 
6. Количество вывозимых порубочных остатков

###Кустарники

In [10]:
def chop_bush_counter(df):
  try:
    df_bush = df[(~df.bush.isna()) & (df.action == 'Вырубить')]\
          [['type', 'bush', 'height', 'description']]
    df_bush['bush_cat'] = df['type'].apply(lambda x: '1поросль' if x == 'поросль' else '2куст')
    df_bush_to_chop = df_bush.groupby('bush_cat', as_index=False)[['bush']].sum()
    return df_bush_to_chop
  except:
    print('Ошибочка')

In [11]:
df_bush_to_chop = chop_bush_counter(df)

In [12]:
df_bush_to_chop

,bush_cat,bush
0,1поросль,845.0
1,2куст,43.0


In [38]:
bush_weigth = df_bush_to_chop.bush[0]*0.025 + df_bush_to_chop.bush[1]*0.05

In [39]:
print(f'Общее количество кустов {df_bush_to_chop.bush.sum()}, вес порубочных остатков {bush_weigth} т.')

Общее количество кустов 888.0, вес порубочных остатков 23.275 т.


###Деревья

Сначала выделим некоторые вспомогательные функции: деление на группы, подсчет ствольности, деление на категории по твердости древесины, и впомогательные данные по весу в тоннах относительно диаметра ствола

In [77]:
#ствольность обычно указана в комментарии
def is_mnogostv(row):
  for i in range(5, 0, -1):
    if str(i)+' с' in row:
        return int(i)
  return 1   

In [78]:
def tree_type(row):
  tverd_types = ['Клен', 'Бук', 'Орех', 'Ясень' , 'Яблоня', 'Вишня', 
           'Сухостой', 'Самосев', 'Рябина', 'Акация']
  for tree_type in tverd_types:
    if tree_type.lower() in row.lower():
      return 'Твердолиственное'
  return 'Мягколиственное/хвойное'

In [79]:
def diameter_bins(row):
  return f'до {row // 10 + 1}0'

In [80]:
df_mass = {'до 10' : 0.0794285714285714, 
           'до 20' : 0.277714285714286, 
           'до 30' : 0.763809523809524, 
           'до 40' : 1.53942857142857, 
           'до 50' : 2.67476190476191, 
           'до 60' : 4.01542857142857, 
           'до 70' : 6.57371428571429, 
           'до 80' : 8.21714285714286, 
           'до 90' : 10.6822857142857, 
           'поросль' : 0.05, 
           'куст' : 0.01}

А теперь собственно функция, которая посчитает нам итоговую таблицу c количеством деревьев по категориям и диаметром и количеством порубочных остатков (а также диаметры пней для корчевки).

In [95]:
def chop_tree_counter(df):
  try:
    df_tree = df[(df.bush.isna()) & (df.action == 'Вырубить')]\
              [['type', 'tree', 'diameter', 'height', 'description']]
    df_tree['stvol'] = df.description.apply(is_mnogostv)
    df_tree['diameter'] = df_tree['diameter'].apply(lambda x: int(str(x).split('-')[-1]))
    df_tree['stvol_count'] = df_tree['stvol'] * df_tree['tree']
    df_tree['stump_diameter'] = df_tree['diameter'] * df_tree['stvol'] + 5
    df_tree['tree_type'] = df_tree['type'].apply(tree_type)
    df_tree['diameter_bins'] = df_tree['diameter'].apply(diameter_bins)
    df_tree['stump_diameter_bins'] = df_tree['stump_diameter'].apply(diameter_bins)
    df_tree['weight'] = df_tree['diameter_bins'].map(df_mass) * df_tree['stvol_count']
    df_tree_grouped = df_tree.groupby(['tree_type', 'diameter_bins'])\
              [['tree', 'stvol_count', 'weight']].sum()
    df_tree_trees = (pd.concat([df_tree_grouped,
           df_tree_grouped.groupby(level=0).sum()
           .assign(claim_type= "total")
           .set_index('claim_type', append=True)]).sort_index()
           .rename(columns={'tree':'Кол-во деревьев', 
                            'stvol_count': 'Кол-во стволов',
                            'weight': 'Тоннаж'}))
    df_tree_grouped_diam = df_tree.groupby('diameter_bins')\
              [['tree', 'stvol_count', 'weight']].sum()
    
    df_stumps = (df_tree.groupby(['tree_type', 'stump_diameter_bins'])[['tree']].sum()
                .rename(columns={'tree':'Кол-во пней'}))
    return df_tree_trees, df_stumps, df_tree_grouped_diam
  except:
    print('Ошибочка')

In [96]:
trees, stumps, trees2 = chop_tree_counter(df)

In [97]:
trees

Кол-во деревьев  Кол-во стволов  \
tree_type               diameter_bins                                    
Мягколиственное/хвойное total                     80.0           121.0   
                        до 10                      4.0             6.0   
                        до 20                     58.0            89.0   
                        до 30                      8.0            10.0   
                        до 40                      7.0            10.0   
                        до 50                      2.0             5.0   
                        до 60                      1.0             1.0   
Твердолиственное        total                    261.0           307.0   
                        до 10                     82.0            84.0   
                        до 20                    108.0           129.0   
                        до 30                     43.0            52.0   
                        до 40                     24.0            37.0   
                        до 50                      4.0             5.0   

                                           Тоннаж  
tree_type               diameter_bins              
Мягколиственное/хвойное total           65.614762  
                        до 10            0.476571  
                        до 20           24.716571  
                        до 30            7.638095  
                        до 40           15.394286  
                        до 50           13.373810  
                        до 60            4.015429  
Твердолиственное        total          152.547905  
                        до 10            6.672000  
                        до 20           35.825143  
                        до 30           39.718095  
                        до 40           56.958857  
                        до 50           13.373810

In [98]:
trees2

,tree,stvol_count,weight
diameter_bins,,,
до 10,86.0,90.0,7.148571
до 20,166.0,218.0,60.541714
до 30,51.0,62.0,47.356190
до 40,31.0,47.0,72.353143
до 50,6.0,10.0,26.747619
до 60,1.0,1.0,4.015429


In [99]:
trees2.stvol_count.sum()

428.0

In [100]:
trees2.tree.sum()

341.0

In [101]:
tree_weigh = trees.query('diameter_bins == "total"')['Тоннаж'].sum()

In [102]:
print(f'''Итого вывозимый тоннаж порубочных остатков 
      {tree_weigh.round(2)} от деревьев, 
      {bush_weigth} от кустов, 
      всего {round(tree_weigh + bush_weigth, 2)} тонны''')

Итого вывозимый тоннаж порубочных остатков 
      218.16 от деревьев, 
      23.275 от кустов, 
      всего 241.44 тонны


In [103]:
stumps

Кол-во пней
tree_type               stump_diameter_bins             
Мягколиственное/хвойное до 140                       1.0
                        до 170                       1.0
                        до 20                       34.0
                        до 30                       22.0
                        до 40                        9.0
                        до 50                        6.0
                        до 60                        3.0
                        до 70                        3.0
                        до 90                        1.0
Твердолиственное        до 10                        3.0
                        до 100                       1.0
                        до 120                       2.0
                        до 140                       1.0
                        до 20                      155.0
                        до 30                       37.0
                        до 40                       37.0
                        до 50                       13.0
                        до 60                        3.0
                        до 70                        5.0
                        до 80                        1.0
                        до 90                        3.0